In [1]:
import os
import sys
import yaml
from typing import (
    Dict,
    List, 
    Optional,
    Tuple,
    Union
)

In [2]:
__file__ = os.path.join(os.getcwd(),"cs_wrap1.ipynb")

In [3]:
mod_path = os.path.join(os.path.dirname(os.path.abspath(__file__)),"..")

In [4]:
sys.path.append(mod_path)

In [5]:
from convert_source.utils.bids_info import (
    construct_bids_dict,
    construct_bids_name
)

In [6]:
from convert_source.utils.utils import (
    BIDSimg,
    collect_info,
    convert_image_data,
    get_metadata,
    list_in_substr,
    SubInfoError,
    SubDataInfo,
)

In [7]:
from convert_source.utils.fileio import TmpDir

In [8]:
from convert_source.utils.const import DEFAULT_CONFIG

In [9]:
from convert_source.imgio.dcmio import (
    is_valid_dcm
)

In [10]:
def read_config(config_file: Optional[str] = "", 
                verbose: Optional[bool] = False
                ) -> Tuple[Dict[str,str],List[str],Dict[str,Union[str,int]]]:
    '''Reads configuration file and creates a dictionary of search terms for 
    certain modalities provided that BIDS modalities are used as keys. If
    exclusions are provided (via the key 'exclude') then an exclusion list is 
    created. Otherwise, 'exclusion_list' is returned as an empty list. If 
    additional settings are specified, they should be done so via the key
    'metadata' to enable writing of additional metadata. Otherwise, an 
    empty dictionary is returned.

    BIDS modalities:
        - anat:
            - T1w, T2w, FLAIR, etc.
        - func:
            - task:
                - resting state, <task-name>
        - dwi
        - fmap

    Usage example:
        >>> [search_dict, exclusion_list, meta_dict] = read_config(config_file)
    
    Arguments:
        config_file: File path to yaml configuration file. If no file is used, then the default configuration file is used.
        verbose: Prints additional information to screen.
    
    Returns: 
        search_dict: Nested dictionary of search terms for BIDS modalities.
        exclusion_list: List of exclusion terms.
        meta_dict: Nested dictionary of metadata terms to write to JSON file(s).
    
    Raises:
        ConfigFileReadError: Error that arises if no heuristic search terms are provided.
    '''
    class ConfigFileReadError(Exception):
        pass

    if config_file:
        config_file: str = os.path.abspath(config_file)
    else:
        config_file: str = DEFAULT_CONFIG

    with open(config_file) as file:
        data_map: Dict[str,str] = yaml.safe_load(file)
        if verbose:
            print("Initialized parameters from configuration file")
    
    # Required search terms
    if any("search" in data_map for element in data_map):
        if verbose:
            print("Categorizing search terms")
        search_dict: Dict[str,str] = data_map["search"]
        del data_map["search"]
    else:
        if verbose:
            print("Heuristic search terms required. Exiting...")
        raise ConfigFileReadError("Heuristic search terms required. Exiting...")

    # Exclusion terms  
    if any("exclude" in data_map for element in data_map):
        if verbose:
            print("Exclusion option implemented")
        exclusion_list: List[str] = data_map["exclude"]
        del data_map["exclude"]
    else:
        if verbose:
            print("Exclusion option not implemented")
        exclusion_list: List = list()
    
    # Metadata terms
    if any("metadata" in data_map for element in data_map):
        if verbose:
            print("Including additional settings for metadata")
        meta_dict: Dict[str,Union[str,int]] = data_map["metadata"]
        del data_map["metadata"]
    else:
        if verbose:
            print("No metadata settings")
        meta_dict: Dict = dict()
        
    return search_dict,exclusion_list,meta_dict

In [11]:
[search_dict,exclusion_list,meta_dict] = read_config()

In [12]:
# search_dict
# exclusion_list
# meta_dict

In [13]:
par_img_dir = "test.img"

In [14]:
par_img_dir = os.path.abspath(par_img_dir)

In [15]:
subs_data = collect_info(parent_dir=par_img_dir,
                        exclusion_list=exclusion_list)

In [16]:
subs_data[90].ses

'001'

In [17]:
# TODO:
#   - Write function that takes input (of array) of SubDataInfo object(s) {sub,data,ses}, and search dict - to search and categorize data
#   - helper/support function to aid in search and categorization of data.

In [10]:
def convert_modality(sub_data: SubDataInfo,
                     out_dir: str,
                     search_dict: Dict[str,str],
                     meta_dict: Optional[Dict[str,Union[int,str]]] = None,
                     keep_unknown: Optional[bool] = False):
    '''working doc-string'''

    mod_found: bool = False

    sub = sub_data.sub
    ses = sub_data.ses
    data = sub_data.data

    if 'dcm' in data:
        if is_valid_dcm(dcm_file=data,raise_exc=False,verbose=False):
            pass
        else:
            return None
    
    for key,item in search_dict.items():
        for dict_key,dict_item in search_dict[key].items():
            if isinstance(dict_item,list):
                if list_in_substr(in_list=dict_item,in_str=data):
                    mod_found = True
                    if verbose:
                        print(f"{key} - {dict_key}: {dict_item}")
                    modality_type = key
                    modality_label = dict_key
                    [com_param_dict, scan_param_dict] = get_metadata(dictionary=meta_dict,modality_type=modality_type)
                    # Conditionals for file conversion
            elif isinstance(dict_item,dict):
                tmp_dict = search_dict[key]
                for d_key,d_item in tmp_dict[dict_key].items():
                    if list_in_substr(in_list=d_item,in_str=data):
                        if verbose:
                            print(f"{key} - {dict_key} - {d_key}: {d_item}")
                        modality_type = key
                        modality_label = dict_key
                        task = d_key
                        [com_param_dict, scan_param_dict] = utils.get_metadata(dictionary=meta_dict,modality_type=modality_type,task=task)
                        # Conditionals for file conversion

In [31]:
# Pass the large number of args as a dict
# 
# Needs to run within for-loop
# 
# Unable to test without actual data
# 
# img_data = convert_image_data(file=subs_data[0].data,
#                                 basename="random_str",
#                                 out_dir="out_dir")

In [19]:
# convert_to_bids
def data_to_bids(sub_data: SubDataInfo,
                 out_dir: str,
                 modality_type: Optional[str] = "",
                 modality_label: Optional[str] = "",
                 meta_dict: Optional[Dict] = {},
                 mod_dict: Optional[Dict] = {}
                 ):
    '''working doc-string'''
    sub_dir = os.path.join(out_dir,"sub-" + sub_data.sub)
    
    if sub_data.ses:
        sub_dir = os.path.join(sub_dir,"ses-" + sub_data.ses)
    
    if sub_dir:
        sub_dir: str = os.path.abspath(sub_dir)
    else:
        os.makedirs(sub_dir)
        sub_dir: str = os.path.abspath(sub_dir)
    
    if '.par' in sub_data.data or '.dcm' in sub_data.data:
        with TmpDir(tmp_dir=sub_data,use_cwd=False) as tmp:
            with TmpDir.TmpFile(tmp_dir=tmp.tmp_dir) as f:
                tmp.mk_tmp_dir()
                [_path, basename, _ext] = f.file_parts()
                img_data = convert_image_data(file=sub_data.data,
                                              basename=basename,
                                              out_dir=tmp.tmp_dir)